# LLM Model Development for DDD Analysis

สมุดบันทึกนี้ใช้สำหรับการพัฒนาและทดสอบโมเดล LLM สำหรับการวิเคราะห์ Domain-Driven Design (DDD) จากข้อกำหนดภาษาไทย

## วัตถุประสงค์
- พัฒนาฟังก์ชันจำลอง LLM ที่จะใช้ในแอพพลิเคชันหลัก
- ทดสอบรูปแบบผลลัพธ์ที่ได้จาก LLM
- ปรับแต่งโมเดลเพื่อให้ผลลัพธ์มีคุณภาพดีขึ้น

In [2]:
# นำเข้าไลบรารี่ที่จำเป็น
import json
import time
import os
import sys

# เพิ่มโฟลเดอร์ src เข้าไปใน path เพื่อที่จะสามารถ import ไฟล์จาก src ได้
sys.path.append('../src')

## ฟังก์ชัน Mock LLM

ฟังก์ชันด้านล่างนี้จำลองการทำงานของ LLM โดยใช้ข้อมูลตัวอย่างที่กำหนดไว้ล่วงหน้า
ในอนาคตเราจะเชื่อมต่อกับ API ของ LLM จริงเพื่อวิเคราะห์ข้อกำหนด

In [3]:
def analyze_with_llm_mock(requirement_text: str) -> dict:
    """
    *** ฟังก์ชันจำลอง (Mock Function) ***
    ฟังก์ชันนี้จำลองการส่งข้อความไปให้ LLM วิเคราะห์และรอรับผลลัพธ์
    ในอนาคต ส่วนนี้จะถูกแทนที่ด้วยการเรียกใช้ API ของ LLM จริง
    """
    print("[INFO] กำลังส่งข้อมูลเพื่อวิเคราะห์ด้วย LLM (จำลอง)...")
    # จำลองดีเลย์ เหมือนกำลังรอผลจาก LLM
    for i in range(3):
        time.sleep(0.5)
        print("...")

    print("[INFO] ได้รับผลลัพธ์จาก LLM (จำลอง) เรียบร้อยแล้ว")

    # --- ผลลัพธ์ตัวอย่างที่ Hardcode ไว้ ---
    # ในการใช้งานจริง ผลลัพธ์นี้จะมาจาก LLM โดยตรง
    mock_response = {
        "boundedContext": "การสั่งซื้อและการจัดส่ง (Ordering & Shipping)",
        "aggregates": [
            {
                "root": "Order",
                "description": "เป็นศูนย์กลางของกระบวนการสั่งซื้อสินค้าทั้งหมด",
                "entities": ["OrderItem"],
                "properties": [
                    {"name": "orderId", "type": "String"},
                    {"name": "customerId", "type": "String"},
                    {"name": "shippingAddress", "type": "Address"},
                    {"name": "totalPrice", "type": "Decimal"},
                ],
            }
        ],
        "entities": [
            {
                "name": "Customer",
                "properties": [
                    {"name": "customerId", "type": "String"},
                    {"name": "customerType", "type": "String", "enum": ["ทั่วไป", "VIP"]},
                ],
            }
        ],
        "businessRules": [
            {
                "ruleId": "SHIP-001",
                "description": "คำนวณค่าจัดส่งสำหรับลูกค้าทั่วไปที่ยอดซื้อไม่ถึง 500 บาท",
                "condition": "customer.customerType == 'ทั่วไป' AND order.totalPrice < 500",
                "action": "order.shippingFee = 40",
            },
            {
                "ruleId": "DISC-001",
                "description": "ให้ส่วนลด 10% และส่งฟรีสำหรับลูกค้า VIP",
                "condition": "customer.customerType == 'VIP'",
                "action": "order.discount = order.totalPrice * 0.10; order.shippingFee = 0",
            },
        ],
    }
    return mock_response

## ทดสอบการทำงานของ Mock LLM

เราจะลองให้โมเดลวิเคราะห์ข้อความทดสอบสั้นๆ และดูผลลัพธ์ที่ได้

In [4]:
# ข้อความตัวอย่างสำหรับทดสอบ
test_requirement = """
# ระบบสั่งซื้อสินค้าออนไลน์
- ลูกค้าสามารถสั่งซื้อสินค้าผ่านเว็บไซต์ได้
- ลูกค้าประเภท VIP จะได้รับส่วนลด 10% และฟรีค่าจัดส่ง
- ลูกค้าทั่วไปที่มียอดสั่งซื้อน้อยกว่า 500 บาท ต้องจ่ายค่าจัดส่ง 40 บาท
"""

# วิเคราะห์ข้อความทดสอบ
result = analyze_with_llm_mock(test_requirement)

# แสดงผลในรูปแบบที่อ่านง่าย
print(json.dumps(result, ensure_ascii=False, indent=2))

[INFO] กำลังส่งข้อมูลเพื่อวิเคราะห์ด้วย LLM (จำลอง)...
...
...
...
[INFO] ได้รับผลลัพธ์จาก LLM (จำลอง) เรียบร้อยแล้ว
{
  "boundedContext": "การสั่งซื้อและการจัดส่ง (Ordering & Shipping)",
  "aggregates": [
    {
      "root": "Order",
      "description": "เป็นศูนย์กลางของกระบวนการสั่งซื้อสินค้าทั้งหมด",
      "entities": [
        "OrderItem"
      ],
      "properties": [
        {
          "name": "orderId",
          "type": "String"
        },
        {
          "name": "customerId",
          "type": "String"
        },
        {
          "name": "shippingAddress",
          "type": "Address"
        },
        {
          "name": "totalPrice",
          "type": "Decimal"
        }
      ]
    }
  ],
  "entities": [
    {
      "name": "Customer",
      "properties": [
        {
          "name": "customerId",
          "type": "String"
        },
        {
          "name": "customerType",
          "type": "String",
          "enum": [
            "ทั่วไป",
            "VIP"

## การปรับแต่งโมเดลและรูปแบบผลลัพธ์

ในส่วนนี้เราสามารถทดลองเพิ่มเติมหรือปรับเปลี่ยนรูปแบบผลลัพธ์ของโมเดล mock เพื่อให้เหมาะสมกับความต้องการมากขึ้น

In [5]:
# ทดลองเพิ่มหมวดหมู่ "valueObjects" เข้าไปในโมเดลจำลอง
def enhanced_analyze_with_llm_mock(requirement_text: str) -> dict:
    # เรียกใช้ฟังก์ชันเดิม
    result = analyze_with_llm_mock(requirement_text)
    
    # เพิ่ม Value Objects เข้าไปในผลลัพธ์
    result["valueObjects"] = [
        {
            "name": "Address",
            "properties": [
                {"name": "street", "type": "String"},
                {"name": "city", "type": "String"},
                {"name": "postalCode", "type": "String"},
                {"name": "country", "type": "String"}
            ]
        },
        {
            "name": "Money",
            "properties": [
                {"name": "amount", "type": "Decimal"},
                {"name": "currency", "type": "String"}
            ]
        }
    ]
    
    return result

# ทดสอบฟังก์ชันที่ปรับปรุงแล้ว
enhanced_result = enhanced_analyze_with_llm_mock(test_requirement)
print(json.dumps(enhanced_result, ensure_ascii=False, indent=2))

[INFO] กำลังส่งข้อมูลเพื่อวิเคราะห์ด้วย LLM (จำลอง)...
...
...
...
[INFO] ได้รับผลลัพธ์จาก LLM (จำลอง) เรียบร้อยแล้ว
{
  "boundedContext": "การสั่งซื้อและการจัดส่ง (Ordering & Shipping)",
  "aggregates": [
    {
      "root": "Order",
      "description": "เป็นศูนย์กลางของกระบวนการสั่งซื้อสินค้าทั้งหมด",
      "entities": [
        "OrderItem"
      ],
      "properties": [
        {
          "name": "orderId",
          "type": "String"
        },
        {
          "name": "customerId",
          "type": "String"
        },
        {
          "name": "shippingAddress",
          "type": "Address"
        },
        {
          "name": "totalPrice",
          "type": "Decimal"
        }
      ]
    }
  ],
  "entities": [
    {
      "name": "Customer",
      "properties": [
        {
          "name": "customerId",
          "type": "String"
        },
        {
          "name": "customerType",
          "type": "String",
          "enum": [
            "ทั่วไป",
            "VIP"

## การแปลงโมเดลจำลองเป็นโค้ดจริง

ในอนาคต เราสามารถปรับเปลี่ยนฟังก์ชันจำลองนี้ให้เชื่อมต่อกับ API ของโมเดล LLM จริง
โดยใช้ไลบรารีต่างๆ เช่น OpenAI API, HuggingFace API หรืออื่นๆ 

ตัวอย่างโค้ดด้านล่างเป็นแนวทางที่อาจใช้ในการเชื่อมต่อกับ OpenAI API (โค้ดนี้เป็นเพียงตัวอย่าง)

In [7]:
# ตัวอย่างการใช้ OpenAI API (ไม่ได้ทำงานจริง - เป็นเพียงตัวอย่าง)
def analyze_with_openai_api(requirement_text: str) -> dict:
    """
    ฟังก์ชันสำหรับวิเคราะห์ข้อกำหนดโดยใช้ OpenAI API
    """
    # นี่เป็นเพียงโค้ดตัวอย่าง - ต้องติดตั้ง openai package และกำหนด API key ก่อน
    # import openai
    # openai.api_key = "your-api-key" 
    
    try:
        print("[INFO] กำลังส่งข้อมูลไปยัง OpenAI API...")
        
        # ตัวอย่างการเรียกใช้ API (ไม่ได้รันจริง)
        """
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "คุณเป็นผู้เชี่ยวชาญด้าน Domain-Driven Design โปรดวิเคราะห์ข้อกำหนดต่อไปนี้และสกัด bounded context, aggregates, entities, value objects, และ business rules ในรูปแบบ JSON"},
                {"role": "user", "content": requirement_text}
            ],
            temperature=0.3,
            max_tokens=2000
        )
        
        # แปลงคำตอบเป็น JSON
        result_str = response.choices[0].message.content
        result = json.loads(result_str)
        """
        
        # สำหรับตัวอย่างนี้ เราจะส่งคืนผลลัพธ์จำลองเดิม
        result = analyze_with_llm_mock(requirement_text)
        
        print("[INFO] ได้รับผลลัพธ์จาก OpenAI API เรียบร้อย")
        return result
        
    except Exception as e:
        print(f"[ERROR] เกิดข้อผิดพลาดในการเรียกใช้ OpenAI API: {e}")
        return None

# หมายเหตุ: ไม่มีการเรียกใช้ฟังก์ชันนี้จริง เนื่องจากเป็นเพียงตัวอย่าง

## สรุป

สมุดบันทึกนี้แสดงให้เห็นถึงการพัฒนาและทดสอบโมเดล LLM จำลองสำหรับการวิเคราะห์ DDD จากข้อกำหนดภาษาไทย

ในอนาคต เราสามารถ:
1. เชื่อมต่อกับ API ของโมเดล LLM จริง
2. ปรับปรุงรูปแบบ prompt เพื่อให้ได้ผลลัพธ์ที่ดีขึ้น
3. เพิ่มการประมวลผลหลังจากได้รับผลลัพธ์จาก LLM เพื่อทำความสะอาดหรือปรับแต่งข้อมูล
4. นำฟังก์ชันที่พัฒนาในสมุดบันทึกนี้ไปใช้ในแอพพลิเคชันหลัก